In [1]:
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from gensim.models import FastText
from keras.models import Sequential

In [2]:
def preprocess_data(file_path):
    texts = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            text, emotion = line.strip().split(';')
            texts.append(text)
            labels.append(emotion)
    return texts, labels

In [3]:
train_texts, train_labels = preprocess_data('data/train.txt')
test_texts, test_labels = preprocess_data('data/test.txt')

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

In [5]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)


In [6]:
max_sequence_length = max(len(seq) for seq in train_sequences)
train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

In [8]:
fasttext_model = FastText(sentences=[text.split() for text in train_texts], window=5, min_count=1, workers=4, sg=1)



In [10]:
train_vectors = np.array([np.mean([fasttext_model.wv[word] for word in text.split() if word in fasttext_model.wv] or [np.zeros(100)], axis=0) for text in train_texts])
test_vectors = np.array([np.mean([fasttext_model.wv[word] for word in text.split() if word in fasttext_model.wv] or [np.zeros(100)], axis=0) for text in test_texts])


In [11]:
label_dict = {
    "sadness": 0,
    "joy": 1,
    "anger": 2,
    "fear": 3,
    "love": 4,
    "surprise": 5
}
train_labels_numeric = np.array([label_dict[label] for label in train_labels])
test_labels_numeric = np.array([label_dict[label] for label in test_labels])

In [13]:
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length),
    Bidirectional(LSTM(32)),
    Dense(6, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 66, 100)           1521300   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 1555745 (5.93 MB)
Trainable params: 1555745 (5.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
model.fit(train_data, train_labels_numeric, validation_split=0.2, epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 7s 27ms/step - loss: -9.9154 - accuracy: 0.3370 - val_loss: -20.1590 - val_accuracy: 0.3262
Epoch 2/10
200/200 [==============================] - 5s 23ms/step - loss: -35.9101 - accuracy: 0.3373 - val_loss: -51.2886 - val_accuracy: 0.3262
Epoch 3/10
200/200 [==============================] - 5s 26ms/step - loss: -84.0163 - accuracy: 0.3373 - val_loss: -122.4532 - val_accuracy: 0.3262
Epoch 4/10
200/200 [==============================] - 5s 25ms/step - loss: -182.7585 - accuracy: 0.3508 - val_loss: -234.6270 - val_accuracy: 0.3909
Epoch 5/10
200/200 [==============================] - 5s 25ms/step - loss: -309.9901 - accuracy: 0.4293 - val_loss: -360.8967 - val_accuracy: 0.4247
Epoch 6/10
200/200 [==============================] - 6s 28ms/step - loss: -457.7295 - accuracy: 0.4620 - val_loss: -517.6733 - val_accuracy: 0.4731
Epoch 7/10
200/200 [==============================] - 5s 23ms/step - loss: -625.8036 - accuracy: 0.4856 - val_lo

In [15]:
loss, accuracy = model.evaluate(test_data, test_labels_numeric)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 1s 3ms/step - loss: -1206.6383 - accuracy: 0.4820
Test Loss: -1206.6383056640625
Test Accuracy: 0.4819999933242798
